In [ ]:
%pip install -r requirements.txt
!python -m spacy download en_core_web_sm

In [ ]:
import symbl

def symbl_get_topics(text: str, video_id: str | None):
    """Gets (abstract) topics keywords"""

    conversation_object = symbl.Text.process({
        "name": video_id,
        "messages": [{"payload": {"content": text}}] 
    }, wait=True)

    data = conversation_object.get_topics()
    return [x["text"] for x in data]

In [ ]:
import textacy
from textacy.extract import keyterms
from textacy import text_stats


def add_topics(data: dict):
    """Parses and adds topics to the video data"""

    # Ignore the first segment, usually introduction.
    txt = " ".join([x["text"] for x in data["transcript"][1:]]).lower()

    try:
        # TODO: The secret is that this always panics, so it's never used.
        # topics = symbl_get_topics(txt, data["video_id"])
        raise RuntimeError()

    except:
        doc = textacy.make_spacy_doc(txt, lang="en_core_web_sm")
        topics = keyterms.textrank(doc, normalize="lemma", topn=20)

    if "metadata" not in data:
        data["metadata"] = {}
    data["metadata"]["topics"] = topics

def add_misc_readability(data: dict):
    
    readability = []

    for ts in data["transcript"]:
        doc = textacy.make_spacy_doc(ts["text"], lang="en_core_web_sm")
        readability.append(text_stats.flesch_kincaid_grade_level(doc))

    if "metadata" not in data:
        data["metadata"] = {}
    data["metadata"]["readability"] = readability

In [ ]:
import json


def main():
    data = json.load(open("testdata.json"))
    for subdata in data:
        del subdata["metadata"]["entropy"]
    json.dump(data, open("new_testdata.json", "w+"))